<a href="https://colab.research.google.com/github/parkhy0106/HolyMoly/blob/master/Tabular_dacon_parking_expect/Parking_test_autogluon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Environment SETUP

##AutoGluonPreVersion

In [ ]:
# Uninstall mkl for faster neural-network training time
!pip uninstall -y mkl
# Upgrade pip to ensure the latest package versions are available
!pip install -U pip
# Upgrade setuptools to be compatible with namespace packages
!pip install -U setuptools
!pip install -U "mxnet<2.0.0"
# Install pre-release, frozen to a particual pre-release for stability
!pip install --pre "autogluon==0.0.16b20201214"
!pip install -U ipykernel

Found existing installation: mkl 2019.0
Uninstalling mkl-2019.0:
  Successfully uninstalled mkl-2019.0
     |████████████████████████████████| 1.6 MB 4.0 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     |████████████████████████████████| 816 kB 1.1 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


     |████████████████████████████████| 46.9 MB 97 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 309 kB 4.5 MB/s 
     |████████████████████████████████| 242 kB 36.3 MB/s 
     |████████████████████████████████| 41 kB 522 kB/s 
     |████████████████████████████████| 101 kB 10.2 MB/s 
     |████████████████████████████████| 786 kB 41.6 MB/s 
     |████████████████████████████████| 964 kB 36.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
ERROR: Some build dependencies for ConfigSpace<=0.4.16 from https://files.pythonhosted.org/packages/3a/f9/685e3cb6e2a87e4989b83950dfbe8cecc49fd158f4ff3d1368dc62014e8d/ConfigSpace-0.4.16.tar.gz#sha256=73c0832a69aae631cccbb22193394176a7c9c127e53b25258444234eb9e867b3 (from autogluon.core==0.0.16b20201214->autogluon==0.0.16b20201214) conflict with t

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##SDV(Need to be restarted and check numpy version which is 1.20)
######_*NOT COMPATIBLE WITH TENSORFLOW AND PYTORCH_*

##Auto preprossesing tool by Kaggler Sensei
######(https://github.com/jeongyoonlee)현재 비활성화됨

In [ ]:
!git clone https://github.com/jeongyoonlee/Kaggler

Cloning into 'Kaggler'...
remote: Enumerating objects: 1468, done.
remote: Counting objects: 100% (233/233), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 1468 (delta 132), reused 116 (delta 65), pack-reused 1235
Receiving objects: 100% (1468/1468), 2.09 MiB | 12.36 MiB/s, done.
Resolving deltas: 100% (925/925), done.


##My Own functions for This competition(굉장히 편협한 코드. 시간이 없어서 모듈화 못 함)

In [ ]:
directory = '/content/drive/MyDrive/mtfk/%~dp0/open/'
index = 'index'
import pandas as pd
train = pd.read_csv(directory+"train.csv",index_col=index)

In [ ]:
NaN=train['occyp_type'][0]

In [ ]:
def adj_col(a):
  a['occyp_type'] = a['occyp_type'].replace(NaN,'others')
  a = a.drop('FLAG_MOBIL',axis=1)
  a['begin_month'] = -a['begin_month']*30
  
  for x in a.index:
    if a['DAYS_EMPLOYED'][x] == 365243:
      a['occyp_type'][x] = 'beksu'
      a["DAYS_EMPLOYED"][x] = 0
  a["DAYS_EMPLOYED"] = -a["DAYS_EMPLOYED"]
  a["DAYS_BIRTH"] = -a["DAYS_BIRTH"]
 
 
  return a

In [ ]:
def split_col(a,b,num):
 trains=[]
 c=[]
 e=[]
 f=[]
 for x in a[b].unique():
  if len(x) >= num:
   print(x)
   trains.append(a[a[b]==x])
   e.append(x)
  else:
   c.append(a[a[b]==x])
   f.append(x)
 d = pd.concat(c)
 trains.append(d)
 e.append(f)
 return (trains, e)

In [ ]:
def models(a):
  d = []
  e = []
  f = []
  for x in range(len(a)):
    b = a[x].sample(int(len(a[x])/10))
    c = a[x].drop(b.index)
    fit_args = {'train_data': c, 'label': 'credit'}    
    d.append(task.fit(**fit_args,eval_metric='MAE'))
    e.append(d[x].evaluate(b))
    d[x].persist_models()
    d[x].refit_full()
    d[x].distill()
    f.append(d[x].leaderboard(b, silent=True))
  return (d,f)

In [ ]:
def fold(a, n):
  b = []
  c = int(len(a)/n)
  for x in range(n-1):
    b.append(a.sample(c))
    a = a.drop(b[x].index)
  b.append(a)
  return b

In [ ]:
def fold_models(a):
  b = a
  d = []
  e = []
  f = []
  for x in range(len(b)):
    c = pd.concat(a).drop(b[x].index)
    print(c)
    fit_args = {'train_data': c, 'label': '등록차량수'}    
    d.append(task.fit(**fit_args,eval_metric='median_absolute_error',output_directory='/contect/Autogluon'))
    e.append(d[x].evaluate(b[x]))
    d[x].persist_models()
    d[x].refit_full()
    #d[x].distill()
    f.append(d[x].leaderboard(b[x], silent=True))
  return (d,f)

In [ ]:
def check_duplicate(df, *args):
  check_list = list(set(df.columns.tolist())- set(args))
  
  # 중복되는 데이터 모두 저장하는 데이터 프레임  [boolean Seires]
  dup_pair_df = df.duplicated(subset=check_list, keep=False) # 모든 중복 샘플을 모두 True 처리 
  
  sum_dup = dup_pair_df.sum()# 중복 데이터 개수 확인
  print(f"{args}을(를) 제외하고 중복되는 데이터 개수는", sum_dup, "입니다")
  

  # 중복이 있을 경우 해당 데이터들의 index 반환
  if dup_pair_df.sum() != 0:
    print("중복 데이터 비율:", round((sum_dup/len(df)*100),2))
    print()
    return df.loc[dup_pair_df].index


In [ ]:
def group_same(df):
  a = 0
  b = []
  c = []
  for x in df['DAYS_BIRTH'].unique():
    for y in df['income_total'].unique():
      a = df[df['DAYS_BIRTH']==x]
      if a.sum !=0:
        pass
      b.append(a[a['income_total']==y])
      #출생일과 수입이 같으면 중복데이터라고 처리하기(너무 많은 값을 검사하면 시간이 오래 걸리고
      #그렇다고 query를 쓰자고 하니 코드 중복이 많아져서 디버깅하는 시간이 부족한 나에게는 모자람.)
      #라고해서 막 짰더니 실행속도가 너무 느려짐
  for x in range(len(b)):
    if len(b[x])!= 0:
      c.append(x)
  d = [] 
  for x in c:
    d.append(b[x])
  return d

#MODULE LOADING

In [ ]:
import pandas as pd
import numpy as np
import copy

In [ ]:
from Autogluon.tabular import TabularPrediction as task

ModuleNotFoundError: ignored

In [ ]:
cd Kaggler

/content/Kaggler


In [ ]:
from kaggler.preprocessing import LabelEncoder
from kaggler.model import AutoLGB

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [ ]:
import kaggler
print(kaggler.__version__)

0.9.13


In [ ]:
cd ..

/content


In [ ]:
directory = '/content/drive/MyDrive/parking/parking_data/'  # directory where you have downloaded the data CSV files from the competition
label_column = 'credit'  # name of target variable to predict in this competition
eval_metric = 'MAE'  # Optional: specify that competition evaluation metric is  Log_loss
output_directory = '/content/drive/MyDrive/parking/results' 
index = 'index'
target_col = '등록차량수'

# DATA PROCESS SECTION

## Data Loading

In [ ]:
train = pd.read_csv(directory+"train.csv")

In [ ]:
test = pd.read_csv(directory+"test.csv")

In [ ]:
sub = pd.read_csv(directory+'sample_submission.csv') 

##Data Cleansing

In [ ]:
train.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0


In [ ]:
train = adj_col(train)

KeyError: ignored

In [ ]:
test = adj_col(test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


nan을 beksu(days_employed가 0인)와 others(고용상태이긴 한 사람들)로 나누어 표기하고 Days_emplyed가 양수(365243)면 0으로 치환 후 부호 양수로 변화
그 외에도 이런저런 처리를 추가할 수 있도록 함수화

### Overlapped data controling

일단 아예 완전히 일치하는 데이터들을 드랍하는 처리를 한 후에

In [ ]:
train.drop_duplicates(inplace=True)

##################################여기서부터 시작#####################################

온전히 일치하는데 credit값이 흔들리는 불안정한 객체들의 갯수를 확인

In [ ]:
train_c = train.copy()

In [ ]:
train = train_c.copy()

In [ ]:
check_duplicate(train_c, '등록차량수')

('등록차량수',)을(를) 제외하고 중복되는 데이터 개수는 0 입니다


In [ ]:
credit_noise = train_c.loc[check_duplicate(train_c, '등록차량수')]
credit_noise.query('income_total==247500.0').query('begin_month==150.0')

('credit',)을(를) 제외하고 중복되는 데이터 개수는 0 입니다


KeyError: ignored

In [ ]:
#train = train.drop(credit_noise.index)

드랍했더니 성능이 소폭 하락했다. 즉슨 무언가 데이터에 중요한 의미가 있다는 뜻.

In [ ]:
credit_noise['credit'].value_counts(normalize=True).mul(100)

2.0    42.947368
1.0    33.052632
0.0    24.000000
Name: credit, dtype: float64

즉슨 너어무 드랍해버리고 싶지만... 신용등급이 너무 높으신 분들이 많다는 것이 문제였다. 즉 이상치라고만 처리하기엔 경향성 분석에 중요한 희귀값인 친구들이라는 뜻으로도 해석될 것 같다...

In [ ]:
a = 0
b = []
c = []
for x in credit_noise['DAYS_BIRTH'].unique():
  for y in credit_noise['income_total'].unique():
    a = credit_noise[credit_noise['DAYS_BIRTH']==x]
    b.append(a[a['income_total']==y])
    #출생일과 수입이 같으면 중복데이터라고 처리하기(너무 많은 값을 검사하면 시간이 오래 걸리고
    #그렇다고 query를 쓰자고 하니 코드 중복이 많아져서 디버깅하는 시간이 부족한 나에게는 모자람.)
for x in range(len(b)):
  if len(b[x])!= 0:
    c.append(x)
d = [] 
for x in c:
  d.append(b[x])

In [ ]:
d[2].credit.value_counts().index

Float64Index([1.0, 2.0], dtype='float64')

In [ ]:
import copy

In [ ]:
m = copy.deepcopy(d)

In [ ]:
d = copy.deepcopy(m)

In [ ]:
for x in range(len(d)):
  for y in reversed(d[x].credit.value_counts().index): #각 행의 credit 별 갯수를 카운트한 인덱스에서
    bnk=0
    mom = 0
    if d[x].credit.value_counts()[y] > bnk: # 각 값의 갯수가 적어도 0보다는 크므로 갯수에 대해서
      bnk = d[x].credit.value_counts()[y] # 전 인덱스의 값보다 더 크게 된다면
      mom = y # 그 행의 값을 받아와서
  d[x]['credit']= mom # 그 행에 대해 일괄적으로 가장 많은 값을 입력하라

최대한 데이터를 긍정적으로 평가해보았더니, 드랍한 것 보다 성능이 안 나왔다. ( 갯수 많은 중복이 있으면 그걸 선택하되, 동일하다면 좋은 쪽 선택 )

최대한 부정적으로 데이터를 평가해보자.(갯수 많은 중복이 있으면 그걸 선택하되, 동일하다면 낮은 쪽 선택 ) -> 성능 소폭 개선됨

In [ ]:
credit_distilled = pd.concat(d)

In [ ]:
for x in credit_distilled.index:
  train.loc[x] = credit_distilled.loc[x]

###이제 begin_month를 기준으로 발생한 노이즈값들을 살펴볼 차례는 스킵~




In [ ]:
begin_noise = train.loc[check_duplicate(train, 'begin_month')]

('begin_month',)을(를) 제외하고 중복되는 데이터 개수는 20564 입니다
중복 데이터 비율: 77.73



In [ ]:
begin_noise_t = test.loc[check_duplicate(test, 'begin_month')]

('begin_month',)을(를) 제외하고 중복되는 데이터 개수는 6877 입니다
중복 데이터 비율: 68.77



In [ ]:
d = group_same(begin_noise)

In [ ]:
e = group_same(begin_noise_t)

In [ ]:
len(d[0])

5

In [ ]:
train['card_num'] = 0

In [ ]:
test['card_num'] = 0

여전히 중복데이터이긴 하나... 발급받은 카드의 갯수가 될 것이라고 가정하고 카드 갯수 feature을 추가해주었다. predictor가 중복데이터간의 연관성을 파악할 수 있게 도와줄 수 있을 것으로 도움을 주기 위해서가 주 목적으로, 중복데이터를 아예 없앴을 때 발생한 문제점을 해결하기 위한 것. -> 문제가 발생했는데, 성능이 대폭 하락한다. 예측 도중 성능은 상승하는데 이유를 잘 모르겠다. 테스트셋과 트레인셋이 다르게 배치 될 여지가 있어서 그런가? 그럴 수 있는건가?

In [ ]:
for x in range(len(d)):
  awef =len(d[x])
  d[x]['card_num'] = awef

In [ ]:
awef = 0
for x in range(len(e)):
  awef =len(e[x])
  e[x]['card_num'] = awef

In [ ]:
begin_distill = pd.concat(d)

In [ ]:
begin_distill_t = pd.concat(e)

In [ ]:
begin_distill.index

Int64Index([    0,  1773, 11402, 17641, 20389,     1,  2199,  5167,  9605,
            12550,
            ...
            24925, 26295, 24997, 25404, 25131, 25681, 25569, 25736, 26328,
            26345],
           dtype='int64', name='index', length=20564)

In [ ]:
train.loc[begin_distill.index] = begin_distill

In [ ]:
test.loc[begin_distill_t.index] = begin_distill_t

###Collecting Extreamly abnormal data

In [ ]:
def abnorm_collec(data):
    ab = []
    df = copy.deepcopy(data)
    
    ab.append(df[(df['family_size']-df['child_num'])<1])    #자식수와 가족수가 같거나 자식수가 더 많음
    
    ab.append(df[df['child_num']>5])                        #자식의 수가 5명보다 많음
    
    wd = df[df['family_type']=='Single / not married']      #싱글인데
    ab.append(wd[wd['family_size']-wd['child_num']>1])      #동거인이 2명 이상임
    
    wd = df[df['family_type']=='Married']                   #결혼했는데
    ab.append(wd[wd['family_size']-wd['child_num']<2])      #동거인이 없음

    wd = df[df['family_type']=='Civil marriage']            #동거한다는데
    ab.append(wd[wd['family_size']-wd['child_num']<2])      #동거인이 없음

    ab.append(df[df['family_size']<1]) #가족 크기가 1보다 작음

    ab.append(df[(df['family_size']-df['child_num'])>3]) #동거를 3명 이상 함


    fs = df[df['family_type']=='Single / not married'][df[df['family_type']=='Single / not married']['child_num']==0]
    #싱글이고
    ab.append(fs[fs['family_size']>1])
    #동거인이 있음

    a = df[df['family_type']=='Single / not married'][df[df['family_type']=='Single / not married']['family_size']>2]
    #싱글이고 가족 크기가 3 이상인데
    ab.append(a[a['reality']== 'N'][a[a['reality']== 'N']['DAYS_EMPLOYED']<2555])
    #직장 근속을 7년 이상을 못 하고 집이 없음

    ab.append(train[train['income_type']=='Pensioner'][train[train['income_type']=='Pensioner']['occyp_type']!='beksu'])
    #일하면서 연금받음

    wd = df[df['occyp_type']=='Drivers']
    a = wd[wd['car']=='N'][wd[wd['car']=='N']['income_type']!='Commercial associate']
    ab.append(a[a['DAYS_EMPLOYED']<200])
    #드라이버인데, 차가 없고 어디 소속되어있지도 않으며 최소고용보장 이상 일하지 못함(6개월)


    abnorm = pd.concat(ab)
    abnorm.drop_duplicates(inplace=True)
    return abnorm
    


In [ ]:
train = train.drop(abnorm_collec(train).index)

In [ ]:
train = train.drop(train[train['begin_month']==0][train[train['begin_month']==0]['credit']==2].index)

### review the result

In [ ]:
train.등록차량수.value_counts(normalize=True)

359.0    0.017097
505.0    0.012158
780.0    0.009498
98.0     0.008739
509.0    0.008739
           ...   
412.0    0.000760
305.0    0.000760
873.0    0.000760
386.0    0.000380
16.0     0.000380
Name: 등록차량수, Length: 354, dtype: float64

In [ ]:
check_duplicate(train, 'credit')

('credit',)을(를) 제외하고 중복되는 데이터 개수는 1425 입니다
중복 데이터 비율: 5.74



Int64Index([    1,     2,    21,    24,    25,    34,    50,    53,    77,
               78,
            ...
            26346, 26354, 26369, 26386, 26391, 26397, 26403, 26430, 26431,
            26432],
           dtype='int64', name='index', length=1425)

In [ ]:
train.drop_duplicates(inplace=True)

오전 11시 38분 중복제거를 하지 않고 최대한 test의 분포에 맞게 해보았다.

In [ ]:
check_duplicate(train, 'credit')

('credit',)을(를) 제외하고 중복되는 데이터 개수는 0 입니다


중복 제거 완료

In [ ]:
len (train_c)-len(train)

719

온전히 중복 데이터를 삭제한 train_c에서 716개 정도의 데이터가 결손되었다.

##Data Preprocessing and distillation with SDV(TVAE, CTGAN, SMOTE) - TVAE SELECTED
######Using data pregenerated because of long fitting interval

데이터를 최대한 정제(cleansing)한 후에 저장한 일종의 synthetic data table이므로 sampling하여 normalizing 하는데 사용

## (Unactivated)Data Recleansing for synthetic data

##Data Spliting and Loading DATASETS for K-FOLD -- 1st way

In [ ]:
trains = fold(train,10)

In [ ]:
a = 0
for x in range(len(trains)):
  a += len(trains[x])
a

2632

In [ ]:
len(train)

NameError: ignored

## Data Spliting by Occyp_type and Apply Synthetic for remove relation caused by data bias. -- 2nd way

In [ ]:
a = train[train['occyp_type']== 'beksu']
b = train.drop(a.index)

In [ ]:
taa = fold(a,5)
tbb = fold(b,5)

#Trainning Model

## K-Fold

In [ ]:
(predictorsa, ldbda) = fold_models(taa)

NameError: ignored

In [ ]:
(predictorsb, ldbdb) = fold_models(tbb)

Beginning AutoGluon training ...
AutoGluon will save models to /contect/Autogluon/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    16034
Train Data Columns: 17
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == float, but few unique label-values observed and label-values can be converted to int).
	3 unique label values:  [2.0, 1.0, 0.0]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Train Data Class Count: 3
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12333.7 MB
	Train Data (Original)  Memory Usage: 9.66 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:

Predictive performance on given dataset: log_loss = 0.7330322758277912


Persisting 7 models in memory. Models will require 3.33% of memory.
Fitting model: NeuralNetMXNet_FULL ...
	53.62s	 = Training runtime
Fitting model: KNeighborsUnif_FULL ...
	0.04s	 = Training runtime
Fitting model: KNeighborsDist_FULL ...
	0.04s	 = Training runtime
Fitting model: RandomForestGini_FULL ...
	6.06s	 = Training runtime
Fitting model: RandomForestEntr_FULL ...
	8.56s	 = Training runtime
Fitting model: ExtraTreesGini_FULL ...
	4.78s	 = Training runtime
Fitting model: ExtraTreesEntr_FULL ...
	5.66s	 = Training runtime
Fitting model: LightGBM_FULL ...
	1.97s	 = Training runtime
Fitting model: LightGBMXT_FULL ...
	2.93s	 = Training runtime
Fitting model: CatBoost_FULL ...
	88.65s	 = Training runtime
Fitting model: XGBoost_FULL ...
	20.74s	 = Training runtime
Fitting model: LightGBMCustom_FULL ...
	4.06s	 = Training runtime
Fitting model: WeightedEnsemble_FULL_L1 ...
	-0.6961	 = Validation log_loss score
	0.67s	 = Training runtime
	0.0s	 = Validation runtime
Beginning AutoGluon

Predictive performance on given dataset: log_loss = 0.7230096966020778


Persisting 6 models in memory. Models will require 1.72% of memory.
Fitting model: NeuralNetMXNet_FULL ...
	42.97s	 = Training runtime
Fitting model: KNeighborsUnif_FULL ...
	0.04s	 = Training runtime
Fitting model: KNeighborsDist_FULL ...
	0.03s	 = Training runtime
Fitting model: RandomForestGini_FULL ...
	6.06s	 = Training runtime
Fitting model: RandomForestEntr_FULL ...
	8.47s	 = Training runtime
Fitting model: ExtraTreesGini_FULL ...
	4.89s	 = Training runtime
Fitting model: ExtraTreesEntr_FULL ...
	5.86s	 = Training runtime
Fitting model: LightGBM_FULL ...
	1.48s	 = Training runtime
Fitting model: LightGBMXT_FULL ...
	2.72s	 = Training runtime
Fitting model: CatBoost_FULL ...
	67.22s	 = Training runtime
Fitting model: XGBoost_FULL ...
	9.54s	 = Training runtime
Fitting model: LightGBMCustom_FULL ...
	2.77s	 = Training runtime
Fitting model: WeightedEnsemble_FULL_L1 ...
	-0.7289	 = Validation log_loss score
	0.61s	 = Training runtime
	0.0s	 = Validation runtime
Beginning AutoGluon 

...

Persisting 8 models in memory. Models will require 1.83% of memory.
Fitting model: NeuralNetMXNet_FULL ...
	52.77s	 = Training runtime
Fitting model: KNeighborsUnif_FULL ...
	0.04s	 = Training runtime
Fitting model: KNeighborsDist_FULL ...
	0.04s	 = Training runtime
Fitting model: RandomForestGini_FULL ...
	6.06s	 = Training runtime
Fitting model: RandomForestEntr_FULL ...
	8.56s	 = Training runtime
Fitting model: ExtraTreesGini_FULL ...
	4.75s	 = Training runtime
Fitting model: ExtraTreesEntr_FULL ...
	5.67s	 = Training runtime
Fitting model: LightGBM_FULL ...
	2.15s	 = Training runtime
Fitting model: LightGBMXT_FULL ...
	3.87s	 = Training runtime
Fitting model: CatBoost_FULL ...
	80.6s	 = Training runtime
Fitting model: XGBoost_FULL ...
	14.15s	 = Training runtime
Fitting model: LightGBMCustom_FULL ...
	9.06s	 = Training runtime
Fitting model: WeightedEnsemble_FULL_L1 ...
	-0.7285	 = Validation log_loss score
	0.6s	 = Training runtime
	0.0s	 = Validation runtime
Beginning AutoGluon t

Predictive performance on given dataset: log_loss = 0.708679134800413


Persisting 7 models in memory. Models will require 3.44% of memory.
Fitting model: NeuralNetMXNet_FULL ...
	49.84s	 = Training runtime
Fitting model: KNeighborsUnif_FULL ...
	0.04s	 = Training runtime
Fitting model: KNeighborsDist_FULL ...
	0.04s	 = Training runtime
Fitting model: RandomForestGini_FULL ...
	6.26s	 = Training runtime
Fitting model: RandomForestEntr_FULL ...
	8.97s	 = Training runtime
Fitting model: ExtraTreesGini_FULL ...
	4.86s	 = Training runtime
Fitting model: ExtraTreesEntr_FULL ...
	5.91s	 = Training runtime
Fitting model: LightGBM_FULL ...
	1.5s	 = Training runtime
Fitting model: LightGBMXT_FULL ...
	3.26s	 = Training runtime
Fitting model: CatBoost_FULL ...
	88.52s	 = Training runtime
Fitting model: XGBoost_FULL ...
	11.87s	 = Training runtime
Fitting model: LightGBMCustom_FULL ...
	3.63s	 = Training runtime
Fitting model: WeightedEnsemble_FULL_L1 ...
	-0.735	 = Validation log_loss score
	0.57s	 = Training runtime
	0.0s	 = Validation runtime
Beginning AutoGluon t

Predictive performance on given dataset: log_loss = 0.7209547537650699


Persisting 8 models in memory. Models will require 1.84% of memory.
Fitting model: NeuralNetMXNet_FULL ...
	54.1s	 = Training runtime
Fitting model: KNeighborsUnif_FULL ...
	0.04s	 = Training runtime
Fitting model: KNeighborsDist_FULL ...
	0.04s	 = Training runtime
Fitting model: RandomForestGini_FULL ...
	6.16s	 = Training runtime
Fitting model: RandomForestEntr_FULL ...
	8.67s	 = Training runtime
Fitting model: ExtraTreesGini_FULL ...
	4.86s	 = Training runtime
Fitting model: ExtraTreesEntr_FULL ...
	5.77s	 = Training runtime
Fitting model: LightGBM_FULL ...
	2.0s	 = Training runtime
Fitting model: LightGBMXT_FULL ...
	3.18s	 = Training runtime
Fitting model: CatBoost_FULL ...
	92.68s	 = Training runtime
Fitting model: XGBoost_FULL ...
	13.84s	 = Training runtime
Fitting model: LightGBMCustom_FULL ...
	3.97s	 = Training runtime
Fitting model: WeightedEnsemble_FULL_L1 ...
	-0.7276	 = Validation log_loss score
	0.8s	 = Training runtime
	0.0s	 = Validation runtime


In [ ]:
(predictors, ldbd) = fold_models(trains)

Beginning AutoGluon training ...
AutoGluon will save models to /contect/Autogluon/
AutoGluon Version:  0.0.16b20201214
Train Data Rows:    2657
Train Data Columns: 14
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (2550.0, 13.0, 556.21039, 430.32873)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12001.04 MB
	Train Data (Original)  Memory Usage: 1.58 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator

       단지코드  총세대수 임대건물구분  ... 도보 10분거리 내 버스정류장 수 단지내주차면수  등록차량수
644   C2644   779     상가  ...               12.0   190.0  177.0
1911  C2486   589    아파트  ...                4.0   515.0  587.0
397   C2034  1486     상가  ...                3.0   405.0  132.0
421   C1109  1957     상가  ...                2.0   351.0  313.0
2052  C2255   560    아파트  ...                2.0   400.0  410.0
...     ...   ...    ...  ...                ...     ...    ...
2932  C2586    90    아파트  ...                3.0    66.0   57.0
2934  C2586    90    아파트  ...                3.0    66.0   57.0
2942  C2020    40    아파트  ...                2.0    25.0   19.0
2947  C2532   239    아파트  ...                1.0   166.0  146.0
2950  C2532   239    아파트  ...                1.0   166.0  146.0

[2657 rows x 15 columns]


	Types of features in original data (raw dtype, special dtypes):
		('float', [])  : 5 | ['전용면적', '공가수', '도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수', '단지내주차면수']
		('int', [])    : 2 | ['총세대수', '전용면적별세대수']
		('object', []) : 7 | ['단지코드', '임대건물구분', '지역', '공급유형', '자격유형', ...]
	Types of features in processed data (raw dtype, special dtypes):
		('category', []) : 7 | ['단지코드', '임대건물구분', '지역', '공급유형', '자격유형', ...]
		('float', [])    : 5 | ['전용면적', '공가수', '도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수', '단지내주차면수']
		('int', [])      : 2 | ['총세대수', '전용면적별세대수']
	0.2s = Fit runtime
	14 features in original data used to generate 14 features in processed data.
	Train Data (Processed) Memory Usage: 0.19 MB (0.0% of available memory)
Data preprocessing and feature engineering runtime = 0.24s ...
AutoGluon will gauge predictive performance using evaluation metric: 'median_absolute_error'
	To change this, specify the eval_metric argument of fit()
AutoGluon will early stop models using eval

[1000]	train_set's l2: 504.682	train_set's median_absolute_error: -9	valid_set's l2: 2321.49	valid_set's median_absolute_error: -17


	-15.4123	 = Validation median_absolute_error score
	4.26s	 = Training runtime
	0.08s	 = Validation runtime
Fitting model: CatBoost ...
	-9.9995	 = Validation median_absolute_error score
	16.73s	 = Training runtime
	0.03s	 = Validation runtime
Fitting model: XGBoost ...
	-3.2093	 = Validation median_absolute_error score
	13.08s	 = Training runtime
	0.1s	 = Validation runtime
Fitting model: NeuralNetMXNet ...
	-16.3	 = Validation median_absolute_error score
	83.36s	 = Training runtime
	0.29s	 = Validation runtime
Fitting model: LightGBMCustom ...
	-3.4957	 = Validation median_absolute_error score
	3.34s	 = Training runtime
	0.05s	 = Validation runtime
Fitting model: WeightedEnsemble_L1 ...
	-0.2883	 = Validation median_absolute_error score
	0.62s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 130.99s ...


...

	-15.3403	 = Validation median_absolute_error score
	3.88s	 = Training runtime
	0.09s	 = Validation runtime
Fitting model: CatBoost ...
	-10.8121	 = Validation median_absolute_error score
	10.21s	 = Training runtime
	0.02s	 = Validation runtime
Fitting model: XGBoost ...
	-4.0533	 = Validation median_absolute_error score
	7.49s	 = Training runtime
	0.06s	 = Validation runtime
Fitting model: NeuralNetMXNet ...
	-19.0579	 = Validation median_absolute_error score
	64.06s	 = Training runtime
	0.3s	 = Validation runtime
Fitting model: LightGBMCustom ...
	-4.4555	 = Validation median_absolute_error score
	2.56s	 = Training runtime
	0.05s	 = Validation runtime
Fitting model: WeightedEnsemble_L1 ...
	-0.4317	 = Validation median_absolute_error score
	0.65s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 97.73s ...


Predictive performance on given dataset: median_absolute_error = 0.75335693359375


Persisting 2 models in memory. Models will require 0.17% of memory.
Fitting model: RandomForestMSE_FULL ...
	2.66s	 = Training runtime
Fitting model: ExtraTreesMSE_FULL ...
	1.27s	 = Training runtime
Fitting model: KNeighborsUnif_FULL ...
	0.01s	 = Training runtime
Fitting model: KNeighborsDist_FULL ...
	0.01s	 = Training runtime
Fitting model: LightGBM_FULL ...
	1.3s	 = Training runtime
Fitting model: LightGBMXT_FULL ...


[1000]	train_set's l2: 360.708	train_set's median_absolute_error: -7


	2.83s	 = Training runtime
Fitting model: CatBoost_FULL ...
	9.33s	 = Training runtime
Fitting model: XGBoost_FULL ...
	8.51s	 = Training runtime
Fitting model: NeuralNetMXNet_FULL ...
	11.81s	 = Training runtime
Fitting model: LightGBMCustom_FULL ...
	1.61s	 = Training runtime
Fitting model: WeightedEnsemble_FULL_L1 ...
	-0.4317	 = Validation median_absolute_error score
	0.0s	 = Training runtime
	0.0s	 = Validation runtime


## Occyp_type

## abnormal data

#Inference Model

## Inferencing each models

In [ ]:
predictors

In [ ]:
ldbd

[                       model  score_test  ...  can_infer  fit_order
 0         ExtraTreesMSE_FULL   -0.123352  ...       True         13
 1   WeightedEnsemble_FULL_L1   -0.123352  ...       True         22
 2              ExtraTreesMSE   -0.353333  ...       True          2
 3        WeightedEnsemble_L1   -0.353333  ...       True         11
 4        LightGBMCustom_FULL   -2.728210  ...       True         21
 5                    XGBoost   -2.926727  ...       True          8
 6       RandomForestMSE_FULL   -3.076660  ...       True         12
 7               XGBoost_FULL   -3.109100  ...       True         19
 8             LightGBMCustom   -3.738327  ...       True         10
 9            RandomForestMSE   -5.630005  ...       True          1
 10             LightGBM_FULL   -6.614502  ...       True         16
 11                  LightGBM   -7.830322  ...       True          5
 12                  CatBoost   -8.804382  ...       True          7
 13             CatBoost_FULL   -8

## ensemble

/content/AutogluonModels


ag-20210520_034147/  ag-20210520_042546/  ag-20210520_050345/
ag-20210520_035217/  ag-20210520_043605/  ag-20210520_051121/
ag-20210520_040246/  ag-20210520_044500/
ag-20210520_041351/  ag-20210520_045555/


In [ ]:
aii = test[test['occyp_tyoe']=='beksu'].index
bii = test.drop(aii).index

In [ ]:
kf = []
for x in range(len(predictorsa)):
  kf.append(predictorsa[x].predict_proba(test.loc[aii], model = ldbda[x]['model'][0], as_pandas=True, as_multiclass=True))

In [ ]:
kf = []
for x in range(len(predictorsb)):
  kf.append(predictorsb[x].predict_proba(test.loc[bii], model = ldbdb[x]['model'][0], as_pandas=True, as_multiclass=True))

In [ ]:
awef = []
for x in range(len(predictors)):
  for y in range(len(ldbd[x])):
    if ldbd[x]['model'][y] == "WeightedEnsemble_FULL_L1" or ldbd[x]['model'][y] == "WeightedEnsemble_L1" or ldbd[x]['model'][y] == "LightGBMCustom_FULL":
      awef.append(y)
      break

In [ ]:
awef

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [ ]:
kf = []
ite = 0
for x in range(len(predictors)):
  print(ldbd[x]['model'][awef[x]])
  if ldbd[x]['model'][awef[x]] == "RandomForestGini_FULL":
    pass
  else:
    kf.append(predictors[x].predict_proba(test, model = ldbd[x]['model'][awef[x]], as_pandas=True, as_multiclass=True))
    ite += 1

WeightedEnsemble_FULL_L1
WeightedEnsemble_FULL_L1
WeightedEnsemble_FULL_L1
WeightedEnsemble_FULL_L1
WeightedEnsemble_FULL_L1
WeightedEnsemble_FULL_L1
WeightedEnsemble_FULL_L1
WeightedEnsemble_FULL_L1
WeightedEnsemble_FULL_L1
WeightedEnsemble_FULL_L1


In [ ]:
kf[0]

0       685.143311
1       714.956665
2       682.993347
3       682.993347
4       680.663330
           ...    
1017    422.940002
1018    246.713333
1019    246.436661
1020    244.610001
1021    247.139999
Name: 등록차량수, Length: 1022, dtype: float32

In [ ]:
test.head(10)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0
5,C1072,754,아파트,경기도,국민임대,51.71,51,14.0,H,43497000,296780,0.0,2.0,683.0
6,C1072,754,아파트,경기도,국민임대,51.96,198,14.0,H,43497000,296780,0.0,2.0,683.0
7,C1072,754,아파트,경기도,국민임대,51.96,67,14.0,H,43497000,296780,0.0,2.0,683.0
8,C1128,1354,아파트,경기도,국민임대,39.79,368,9.0,H,22830000,189840,0.0,3.0,1216.0
9,C1128,1354,아파트,경기도,국민임대,39.79,30,9.0,H,22830000,189840,0.0,3.0,1216.0


In [ ]:
len(test)/8

127.75

In [ ]:
test['단지코드'][1021]

'C2189'

In [ ]:
sub['code'][149]

'C2189'

In [ ]:
a = copy.deepcopy(sub)

In [ ]:
a = []
res = 0
reslt = []
ress = []
for i in range(len(kf)):
  reslt = []
  b = 0
  for x in range(len(sub)):
    res = 0
    if (test['단지코드'][b]) == (sub['code'][x]):
        a = test['단지코드'][b]
        k = 0
        while(a == sub['code'][x]):
          print(sub['code'][x])
          k += 1
          try:
            res += kf[i][b]
          except KeyError:
            break
          a = test['단지코드'][b]
          b += 1
        res = res/k
    reslt.append(res)
  ress.append(reslt)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2676
C2318
C2318
C1603
C1603
C1603
C1603
C2523
C2523
C2523
C2177
C2177
C2177
C2177
C2177
C2177
C2177
C2177
C2177
C2177
C2177
C2177
C2177
C2177
C2177
C2177
C2177
C2177
C2177
C2177
C2177
C2177
C2177
C2177
C2177
C2411
C2411
C2411
C2411
C2411
C2033
C2033
C2033
C2033
C2033
C2033
C2033
C2033
C2033
C2033
C2033
C2033
C2033
C2033
C2033
C2033
C2033
C2033
C2033
C2033
C2033
C2033
C2033
C2033
C2033
C1604
C1604
C2535
C2535
C1040
C1040
C2417
C2417
C1812
C1812
C1812
C1812
C1812
C1812
C1812
C1812
C1812
C1812
C1812
C1812
C1812
C1812
C1812
C1812
C1812
C1812
C1812
C1812
C1812
C1812
C1812
C1812
C1812
C1812
C1812
C2253
C2253
C2253
C2253
C2253
C2253
C2253
C2253
C2253
C2253
C2253
C2253
C2253
C2253
C2253
C2253
C225

In [ ]:
len(reslt)

150

In [ ]:
a

'C2189'

In [ ]:
mod = [0 for i in range(150)]
for x in range(len(ress)):
  for y in range(len(ress[x])):
    mod[y] += ress[x][y]

In [ ]:
for y in range(len(mod)):
  mod[y] = mod[y] / len(ress)

In [ ]:
mod

[737.0935628255208,
 1076.012983534071,
 493.61684841579853,
 664.7645660400391,
 1160.7104187011719,
 1560.1948464133525,
 840.3144404093424,
 399.39866943359374,
 361.31126770019534,
 330.36424713134767,
 430.48266662597655,
 201.06641755280674,
 390.2446594238281,
 361.7708377838135,
 241.94949241306477,
 350.9586669921875,
 428.7071678161621,
 297.8788859049479,
 236.0751182250977,
 692.2464672851563,
 292.3680612792969,
 424.0683288574219,
 466.35333251953125,
 422.80833435058594,
 266.7256713867188,
 163.948883621781,
 180.19791141633067,
 476.4692220052083,
 460.3159912109375,
 690.5209930419921,
 656.9442209879558,
 326.6933288574219,
 387.137440999349,
 185.45333099365234,
 105.76039978027345,
 318.459055074056,
 462.52233734130857,
 597.7875508626302,
 828.0953254699707,
 334.8012496948242,
 421.0149377441406,
 427.5734014892578,
 432.73979919433594,
 547.9643423897879,
 842.7595872667101,
 1190.5018579101563,
 478.3321966552734,
 598.6135803222656,
 391.5619232177734,
 456.4

In [ ]:
sub['num'] = mod

In [ ]:
sub['num']

0       737.093563
1      1076.012984
2       493.616848
3       664.764566
4      1160.710419
          ...     
145     316.119164
146     412.228665
147     242.290003
148     410.840297
149     184.546665
Name: num, Length: 150, dtype: float64

In [ ]:
kf = []
for x in range(len(predictors)):
  kf.append(predictors[x].predict_proba(test, model = ldbd[x]['model'][0], as_pandas=True, as_multiclass=True))

In [ ]:
test_t = abnorm_collec(test)

Int64Index([31375, 34703, 26701, 27141, 27273, 30423, 30486, 30562, 31647,
            32065, 32462, 34224, 35699, 26680, 29157, 30180, 30211, 31211,
            36021],
           dtype='int64', name='index')

In [ ]:
a = 0
for x in range(ite):
  a += kf[x]

NameError: ignored

In [ ]:
a

,code,num
0,C1072,0
1,C1128,0
2,C1456,0
3,C1840,0
4,C1332,0
...,...,...
145,C2456,0
146,C1266,0
147,C2152,0
148,C1267,0


In [ ]:
k = a.sort_index()
k.head(10)

# FINISHED!!!!

In [ ]:
cd /content/drive/MyDrive/parking/results

/content/drive/MyDrive/parking/results


In [ ]:
a = sub.copy()

In [ ]:
sub.to_csv('Just_tried_mae_10.csv',index = False)

In [ ]:
sub['0'] = k[0]
sub['1'] = k[1]
sub['2'] = k[2]

In [ ]:
k = (sub + a)/ite

In [ ]:
abn = k.loc[test_t.index]

In [ ]:
for x in range(len(k.loc[test_t.index])):
  print(x)
  abn.iloc[x]['0'] = 0.00
  abn.iloc[x]['1'] = 0.00
  abn.iloc[x]['2'] = 1.00
abn.iloc[x]

0
1
2
3
4
5
6
7
8
9
10
11
12


0    0.0
1    0.0
2    1.0
Name: 35699, dtype: float64

In [ ]:
k.loc[abn.index] = abn

In [ ]:
k.loc[test_t.index]

,0,1,2
index,,,
31375,0.0,0.0,1.0
34703,0.0,0.0,1.0
26701,0.0,0.0,1.0
27141,0.0,0.0,1.0
27273,0.0,0.0,1.0
30423,0.0,0.0,1.0
30486,0.0,0.0,1.0
30562,0.0,0.0,1.0
31647,0.0,0.0,1.0


In [ ]:
k.head(10)

,0,1,2
index,,,
26457,0.050738,0.132697,0.816564
26458,0.296015,0.196230,0.507755
26459,0.057182,0.103590,0.839228
26460,0.153849,0.141430,0.704720
26461,0.079917,0.169419,0.750665
26462,0.072972,0.119047,0.807981
26463,0.346094,0.646378,0.007528
26464,0.106634,0.121895,0.771471
26465,0.067727,0.164819,0.767455


In [ ]:
k.to_csv('K_fold_auto_10times_reshape_log_loss.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
a

,0,1,2
index,,,
26457,0,0,0
26458,0,0,0
26459,0,0,0
26460,0,0,0
26461,0,0,0
...,...,...,...
36452,0,0,0
36453,0,0,0
36454,0,0,0


In [ ]:
a = pd.read_csv(output_directory+"/K_fold_auto_13times.csv",index_col=index)
b = pd.read_csv(output_directory+"/Old_beksu.csv",index_col=index)
c = pd.read_csv(output_directory+"/Auto_gluon.csv",index_col=index)
d = pd.read_csv(output_directory+"/Longest_iter.csv",index_col=index)

In [ ]:
e = (a + b + c + d + k)/5

In [ ]:
e.to_csv("Parking_area.csv")